In [2]:
import numpy as np
import pandas as pd

ModuleNotFoundError: No module named 'numpy'

In [6]:
pd.read_csv("movie_review.csv",encoding= 'unicode_escape')

,Review,Polarity
0,tristar / 1 : 30 / 1997 / r ( language violen...,0
1,arlington road 1/4 . directed by mark pellingt...,0
2,the brady bunch movie is less a motion picture...,0
3,janeane garofalo in a romantic comedy -- it wa...,0
4,i'm going to keep this plot summary brief som...,0
...,...,...
195,release date : january 15 1999 starring : sea...,1
196,director : warren beatty || screenplay : warre...,1
197,indie filmmaker marcus van bavel may not have ...,1
198,what surprises me most about the long-awaited ...,1
